In [1]:
#test to add standard scaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# %matplotlib inline
red_wine = pd.read_csv('winequality-red.csv', delimiter=';')
white_wine = pd.read_csv('winequality-white.csv', delimiter=';')


In [3]:
red_wine['class'] = 'red'
white_wine['class'] = 'white'
wine_data = pd.concat([red_wine, white_wine], axis=0, ignore_index=True)


In [4]:
# wine_data.isnull().sum()
# wine_data.dtypes

In [5]:
grade = [] #add column grade to classify the quality of the wine
for i in wine_data['quality']: 
    if i >= 7:
        i = 1
        grade.append(i)
    else:
        i = 0
        grade.append(i)
wine_data['grade'] = grade

In [6]:
wine_data.drop('class', axis=1, inplace=True)
wine_data.drop('quality', axis = 1, inplace = True)

In [7]:
# convert 'Class' feature into numeric
# wine_data.head(10)

In [8]:
# plt.figure(figsize=(15,10))
# sns.heatmap(wine_data.corr(), annot=True, vmin=-1, vmax=1, cmap='PiYG')
# plt.show()

In [9]:
# fig, axes = plt.subplots(2,3, figsize=(18,10))
# sns.scatterplot(data=wine_data, x='alcohol', y='density', hue='class', ax=axes[0, 0])
# sns.scatterplot(data=wine_data, x='free sulfur dioxide', y='total sulfur dioxide', hue='class', ax=axes[0, 1])
# sns.scatterplot(data=wine_data, x='alcohol', y='quality', hue='class', ax=axes[0, 2])
# sns.scatterplot(data=wine_data, x='residual sugar', y='density', hue='class', ax=axes[1, 0])
# sns.scatterplot(data=wine_data, x='volatile acidity', y='total sulfur dioxide', hue='class', ax=axes[1, 1])
# sns.scatterplot(data=wine_data, x='alcohol', y='density', hue='class', ax=axes[1, 2])
# plt.show()

In [10]:
#convert data type to numeric
wine_data = wine_data.astype('float64')
wine_data['grade'] = wine_data['grade'].astype('int64')
data_X = wine_data.drop('grade', axis=1)
data_y = wine_data['grade']

In [11]:
# wine_data.describe()

In [12]:
def train_test_split(X,y,test_size):
    test_size = 0.2
    train_size = 1 - float(test_size)
    total_rows = wine_data.shape[0]
    split =  int(total_rows * (train_size))
    X_train = data_X[0:split]
    X_test = data_X[split:]
    y_train = data_y[0:split]
    y_test = data_y[split:]
    return X_train, X_test, y_train, y_test

In [13]:
X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.2)

In [14]:
y_train[y_train == 0] = -1
y_test[y_test == 0] = -1

In [15]:
print("X train:",len(X_train))
print("X test:",len(X_test))
print("y train:",len(y_train))
print("y test:",len(y_test))

X train: 5197
X test: 1300
y train: 5197
y test: 1300


In [16]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

In [17]:
# implement Support Vector Machines
import numpy as np

class SVM:
    def __init__(self, learning_rate=10000, lambda_param=0.00001, num_iterations=100):
        self.learning_rate = learning_rate
        self.lambda_param = lambda_param
        self.num_iterations = num_iterations
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.num_iterations):
            for idx, x_i in enumerate(X):
                condition = y[idx] * (np.dot(x_i, self.weights) - self.bias) >= 1
                if condition:
                    self.weights -= self.learning_rate * (2 * self.lambda_param * self.weights)
                else:
                    self.weights -= self.learning_rate * (2 * self.lambda_param * self.weights - np.dot(x_i, y[idx]))
                    self.bias -= self.learning_rate * y[idx]

    def predict(self, X):
        approx = np.dot(X, self.weights) - self.bias
        return np.sign(approx)


In [18]:
svm_model = SVM()
svm_model.fit(X_train.to_numpy(), y_train.to_numpy())

In [19]:
# Evaluate accuracy
svm_y_pred = svm_model.predict(X_test.to_numpy())
accuracy = np.mean(svm_y_pred == y_test)
print('Accuracy Score:', float(accuracy*100),"%")

Accuracy Score: 81.0 %


In [20]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(svm_y_pred, y_test)
cm_wine_data = pd.DataFrame(cm)
cm_wine_data.columns = ['Predict -1','Predict 1']
cm_wine_data = cm_wine_data.rename(index={0: 'Actual -1',1:'Actual 1'})
cm_wine_data

,Predict -1,Predict 1
Actual -1,1053,247
Actual 1,0,0


In [21]:
# Future Predictions
new_wine_data = wine_data.sample(5)
new_wine_data


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,grade
3163,9.1,0.28,0.49,2.00,0.059,10.0,112.0,0.99580,3.15,0.46,10.1,-1
3281,7.2,0.25,0.28,14.40,0.055,55.0,205.0,0.99860,3.12,0.38,9.0,1
373,7.4,0.55,0.22,2.20,0.106,12.0,72.0,0.99590,3.05,0.63,9.2,-1
1917,5.9,0.30,0.47,7.85,0.030,19.0,133.0,0.99330,3.52,0.43,11.5,1
4187,6.8,0.18,0.30,12.80,0.062,19.0,171.0,0.99808,3.00,0.52,9.0,1


In [22]:
new_X = new_wine_data.drop('grade', axis=1)

In [23]:
# scaler = StandardScaler()
# new_X = scaler.fit_transform(new_X)

In [24]:
new_X

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
3163,9.1,0.28,0.49,2.00,0.059,10.0,112.0,0.99580,3.15,0.46,10.1
3281,7.2,0.25,0.28,14.40,0.055,55.0,205.0,0.99860,3.12,0.38,9.0
373,7.4,0.55,0.22,2.20,0.106,12.0,72.0,0.99590,3.05,0.63,9.2
1917,5.9,0.30,0.47,7.85,0.030,19.0,133.0,0.99330,3.52,0.43,11.5
4187,6.8,0.18,0.30,12.80,0.062,19.0,171.0,0.99808,3.00,0.52,9.0


In [25]:
svm_y_pred_new = svm_model.predict(new_X)

svm_y_pred_new

array([-1., -1., -1., -1., -1.])